In [411]:
#Import necessary libraries
import numpy as np #This works with numbers
import pandas as pd #This allows us to work with a dataset
from sklearn.model_selection import train_test_split #For splitting data for training and test
from sklearn.preprocessing import StandardScaler #For scaling features
from sklearn import datasets, svm 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score #For metrics and evaluation algorithm
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier #For KNN
from sklearn import tree

In [412]:
#load data into dataframe
data = pd.read_csv('HCV-Egy-Data.csv')

# Remove Outliers 
X = data
col = list(X.drop('Baselinehistological staging',axis=1).columns)
for i in col:
    y = data[i]
    removed_outliers = y.between(y.quantile(.02), y.quantile(.98))
    index_names = data[~removed_outliers].index 
    data.drop(index_names, inplace=True)
    
#dropping histoligical baseline due to duplicate representation of data
data = data.drop('Baseline histological Grading', axis = 1)

In [413]:
#seperate X and y
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [414]:
print(X.head(5))

   Age  Gender  BMI  Fever  Nausea/Vomting  Headache  Diarrhea  \
3   49       2   33      1               2         1         2   
5   58       2   22      2               2         2         1   
6   42       2   26      1               1         2         2   
7   48       2   30      1               1         2         2   
9   45       1   30      2               1         2         2   

   Fatigue & generalized bone ache  Jaundice  Epigastric pain  ...  ALT 12  \
3                                1         2                1  ...      80   
5                                2         2                1  ...      96   
6                                2         2                2  ...     118   
7                                1         1                2  ...     127   
9                                1         1                2  ...     127   

   ALT 24  ALT 36  ALT 48  ALT after 24 w  RNA Base    RNA 4  RNA 12  RNA EOT  \
3      88      48      77              33   1041941  

In [415]:
#Feature selection and engineering
#Calculate biomarker scores
fib4 = (X['Age']*X['AST1'])/(X['Plat']/1000*np.sqrt(X['ALT1']))
apri = 100*(X['AST1']/40)/(X['Plat']/1000)
liverDam = X['AST1']/X['ALT1']

#Feature selection Age, Gender, BMI
#ModX holds features to train classifier
ModX = X.iloc[:,0:3]

#Select ALT scores over time
ModX['ALT1'] = X['ALT1']
ModX['ALT4'] = X['ALT4']
ModX['ALT 24'] = X['ALT 24']
ModX['ALT 48'] = X['ALT 48']

ModX['AST1'] = X['AST1']

ModX['fib4'] = fib4
ModX['apri'] = apri
ModX['liverDam'] = liverDam

ModX

<ipython-input-415-7f38ec5e5516>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ModX['ALT1'] = X['ALT1']
<ipython-input-415-7f38ec5e5516>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ModX['ALT4'] = X['ALT4']
<ipython-input-415-7f38ec5e5516>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

,Age,Gender,BMI,ALT1,ALT4,ALT 24,ALT 48,AST1,fib4,apri,liverDam
3,49,2,33,64,109,88,77,43,1.798309,0.734004,0.671875
5,58,2,22,104,121,65,114,66,2.860413,1.257354,0.634615
6,42,2,26,57,113,107,80,78,2.447898,1.100073,1.368421
7,48,2,30,112,80,45,53,119,2.496731,1.376193,1.062500
9,45,1,30,68,72,81,43,55,3.056393,1.400204,0.808824
...,...,...,...,...,...,...,...,...,...,...,...
1369,32,1,26,98,102,86,110,77,1.776714,1.374107,0.785714
1373,46,1,27,80,64,92,79,118,5.641566,2.742375,1.475000
1376,46,1,27,49,77,103,120,40,2.176384,0.827972,0.816327
1377,52,2,25,50,98,50,122,109,4.379241,1.488746,2.180000


In [416]:
#Apply discretization to calculated ratios
#Modify apri ratio
for i in range(len(X)):
    current = ModX.iloc[i,-2]
    if current > 0.7 :
        ModX.iloc[i,-2] = 1
    else:
        ModX.iloc[i,-2] = 0

#Modify liverdam ratio
for i in range(len(X)):
    current = ModX.iloc[i,-1]
    if current >= 1 :
        ModX.iloc[i,-1] = 1
    else:
        ModX.iloc[i,-1] = 0

#Modify fib4  ratio
for i in range(len(X)):
    current = ModX.iloc[i,-3]
    if current < 1.45 :
        ModX.iloc[i,-3] = 0
    else:
        ModX.iloc[i,-3] = 1
        
print(ModX.head(5))

c:\users\alexa\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


   Age  Gender  BMI  ALT1  ALT4  ALT 24  ALT 48  AST1  fib4  apri  liverDam
3   49       2   33    64   109      88      77    43   1.0   1.0       0.0
5   58       2   22   104   121      65     114    66   1.0   1.0       0.0
6   42       2   26    57   113     107      80    78   1.0   1.0       1.0
7   48       2   30   112    80      45      53   119   1.0   1.0       1.0
9   45       1   30    68    72      81      43    55   1.0   1.0       0.0


In [417]:
#Transform y to only focus on F4
for i in range(len(y)):
    current = y.iloc[i]
    if current != 4:
        y.iloc[i] = 0
        
print(y.head(5))

3    0
5    4
6    4
7    0
9    0
Name: Baselinehistological staging, dtype: int64


In [418]:
#Split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(ModX, y, test_size = 0.20, random_state = 0)

#Feature scale so that one feature doesn't have more influence than another
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [419]:
#Here we are going to make a simple SVM classifier
classifier = svm.NuSVC(nu = 0.54, kernel='rbf')

In [420]:
#train the classifier on train set and then print out its overall accuracy for the train data
classifier.fit(X_train, y_train)

print(classifier.score(X_train,y_train))

0.7660818713450293


In [421]:
#Determine the overall accuracy score of the test data set
print(classifier.score(X_test,y_test))

0.7485380116959064


In [422]:
#Making predictions
y_pred = classifier.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[128   3]
 [ 40   0]]
              precision    recall  f1-score   support

           0       0.76      0.98      0.86       131
           4       0.00      0.00      0.00        40

    accuracy                           0.75       171
   macro avg       0.38      0.49      0.43       171
weighted avg       0.58      0.75      0.66       171



In [423]:
#Training with k-nearest neighbor (KNN)
classifier2 = KNeighborsClassifier(n_neighbors = 10)
classifier2.fit(X_train, y_train)
#Making predictions
y_pred = classifier2.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

The confusion matrix is:
[[127   4]
 [ 40   0]]
              precision    recall  f1-score   support

           0       0.76      0.97      0.85       131
           4       0.00      0.00      0.00        40

    accuracy                           0.74       171
   macro avg       0.38      0.48      0.43       171
weighted avg       0.58      0.74      0.65       171



In [424]:
#Decision trees
#Here we are going to make a simple decision classifier
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
#Determine the overall accuracy score of the test data set
print(clf.score(X_test,y_test))
#Making predictions
y_pred = clf.predict(X_test)
#To evaluate algorithm we will print the confusion matrix and other metrics
print("The confusion matrix is:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

1.0
0.5847953216374269
The confusion matrix is:
[[89 42]
 [29 11]]
              precision    recall  f1-score   support

           0       0.75      0.68      0.71       131
           4       0.21      0.28      0.24        40

    accuracy                           0.58       171
   macro avg       0.48      0.48      0.48       171
weighted avg       0.63      0.58      0.60       171

